# City : Tokyo

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv

# 設置 Selenium 驅動
options = Options()
options.add_argument("--headless")  # 如果需要顯示瀏覽器，請去掉此行
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 設置 CSV 檔案
csv_file = open('google_flights_data_tokyo.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['出發時間', '抵達時間', '出發機場', '抵達機場', '航空公司名稱', '飛行時間', '價格'])

try:
    # 打開 Google Travel 網站
    url = 'https://www.google.com/travel/explore?tfs=CBwQAxoqEgoyMDI0LTA5LTE4KABqDAgCEggvbS8wZnRreHIMCAQSCC9tLzAyajcxQAFIAXABggELCP___________wGYAQKyAQQYASAB&tfu=GgA&authuser=0'
    driver.get(url)

    # 等待 City 連結並點擊
    city_link = WebDriverWait(driver, 30).until(
        EC.visibility_of_element_located((By.XPATH, '//li[contains(@class, "lPyEac") and contains(@class, "P0ukfb") and @role="button"]'))
    )
    city_link.click()
    print("Clicked on TOKYO link")
    
    # 等待“查看航班”按鈕並點擊
    try:
        view_flights_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="yDmH0d"]/c-wiz[2]/div/div[2]/c-wiz/div[2]/div/div/div[2]/div/div[4]/div[1]/div/div[2]/div[4]/div/div/div/a'))
        )
        view_flights_button.click()
        print("Clicked on View Flights link")
        
        # 切換到新分頁
        WebDriverWait(driver, 30).until(lambda driver: len(driver.window_handles) > 1)
        new_tab = driver.window_handles[1]
        driver.switch_to.window(new_tab)
        print("Switched to new tab")
    except TimeoutException:
        print("未找到查看航班按鈕或點擊失敗")
        driver.quit()
        exit()
          
    # 確保頁面元素存在後再查找
    try:
        flight_elements = WebDriverWait(driver, 60).until(
            EC.presence_of_all_elements_located((By.XPATH, "//li[@class='pIav2d']"))
        )
        print(f"找到 {len(flight_elements)} 個航班元素")

        
        for index, flight in enumerate(flight_elements, start=1):
            print(f"處理第 {index} 個航班")

            try:
                # 提取出發時間
                departure_element = flight.find_element(By.XPATH, './/div[@class="wtdjmc YMlIz ogfYpf tPgKwe"]')
                aria_label = departure_element.get_attribute('aria-label')
                # print(f"出發時間 aria-label: {aria_label}")  # 調試輸出
                if '出發時間：' in aria_label:
                    departure_time = aria_label.split('出發時間：')[1].strip('。')
                else:
                    departure_time = 'N/A'
            except NoSuchElementException:
                departure_time = 'N/A'
                
            try:
                # 提取抵達時間（假設 HTML 中的結構相似）
                arrival_element = flight.find_element(By.XPATH, './/div[@class="XWcVob YMlIz ogfYpf tPgKwe"]')
                aria_label = arrival_element.get_attribute('aria-label')
                # print(f"抵達時間 aria-label: {aria_label}")  # 調試輸出
                if '抵達時間：' in aria_label:
                    arrival_time = aria_label.split('抵達時間：')[1].strip('。')
                else:
                    arrival_time = 'N/A'
            except NoSuchElementException:
                arrival_time = 'N/A'
        
            try:
                departure_airport = flight.find_element(By.XPATH, './/div[@class="QylvBf"][1]/span').text
            except NoSuchElementException:
                departure_airport = 'N/A'
        
            try:
                arrival_airport = flight.find_element(By.XPATH, './/div[@class="QylvBf"][2]/span').text
            except NoSuchElementException:
                arrival_airport = 'N/A'
        
            try:
                airline_name = flight.find_element(By.XPATH, './/div[contains(@class, "sSHqwe")]//span').text
            except NoSuchElementException:
                airline_name = 'N/A'
        
            try:
                flight_time = flight.find_element(By.XPATH, './/div[contains(@class, "gvkrdb")]').text
            except NoSuchElementException:
                flight_time = 'N/A'
        
            try:
                price = flight.find_element(By.XPATH, './/div[contains(@class, "FpEdX")]//span').text
            except NoSuchElementException:
                price = 'N/A'
                
            print(f"出發時間: {departure_time}")
            print(f"抵達時間: {arrival_time}")
            print(f"出發機場: {departure_airport}")
            print(f"抵達機場: {arrival_airport}")
            print(f"航空公司名稱: {airline_name}")
            print(f"飛行時間: {flight_time}")
            print(f"價格: {price}")
            print('-' * 40)
            
            # 將數據寫入 CSV 文件
            csv_writer.writerow([departure_time, arrival_time, departure_airport, arrival_airport, airline_name, flight_time, price])
            
    except TimeoutException:
        print("未找到航班元素")
                        
finally:
    driver.quit()
    csv_file.close()


Clicked on TOKYO link
Clicked on View Flights link
Switched to new tab
找到 14 個航班元素
處理第 1 個航班
出發時間: 中午12:10
抵達時間: 下午4:30
出發機場: TPE
抵達機場: NRT
航空公司名稱: 泰國獅航
飛行時間: 3 小時 20 分鐘
價格: $4,897
----------------------------------------
處理第 2 個航班
出發時間: 下午3:40
抵達時間: 晚上8:05
出發機場: TPE
抵達機場: NRT
航空公司名稱: 日本航空
飛行時間: 3 小時 25 分鐘
價格: $7,855
----------------------------------------
處理第 3 個航班
出發時間: 下午2:35
抵達時間: 下午6:55
出發機場: TPE
抵達機場: NRT
航空公司名稱: 中華航空
飛行時間: 3 小時 20 分鐘
價格: $8,098
----------------------------------------
處理第 4 個航班
出發時間: 下午3:10
抵達時間: 晚上7:25
出發機場: TPE
抵達機場: NRT
航空公司名稱: 星宇航空
飛行時間: 3 小時 15 分鐘
價格: $8,429
----------------------------------------
處理第 5 個航班
出發時間: 下午3:30
抵達時間: 晚上7:55
出發機場: TPE
抵達機場: NRT
航空公司名稱: 台灣虎航
飛行時間: 3 小時 25 分鐘
價格: $7,799
----------------------------------------
處理第 6 個航班
出發時間: 下午4:20
抵達時間: 晚上8:35
出發機場: TPE
抵達機場: NRT
航空公司名稱: 中華航空
飛行時間: 3 小時 15 分鐘
價格: $8,098
----------------------------------------
處理第 7 個航班
出發時間: 下午6:05
抵達時間: 晚上10:05
出發機場: TSA
抵達機場: HND
航空公司名稱: 中華航空
飛行時間: 3 小時
價格: $8,